## Panorama Stitching in OpenCV

---

Dr.-Ing. Antje Muntzinger, Hochschule für Technik Stuttgart

antje.muntzinger@hft-stuttgart.de

---

In this notebook, we will stitch a panorama from single images in OpenCV. We will explore two methods: a fast method using OpenCV's `Stitcher` class that handles most intermediate steps automatically, and a more hands-on method where we implement the different steps one by one.

In [ ]:
# install required packages specified in pipfile
!pipenv install

In [ ]:
# imports
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec
%matplotlib inline

import numpy as np 
import cv2

Task 1: Image Preparation
=

**TODO**: 1a) Load two or more overlapping images that you want to stitch. You can use the provided images, but it is highly encouraged to take some overlapping photos yourself. Note that not all photos can successfully be stitched together without modifications, so in case you encounter problems, try the provided images first. **(1 point)**

In [ ]:
##### TODO: adapt n_images to your number of images, and adapt the paths below to load your images
# single images named 1.jpg, 2.jpg etc.
n_images = 3

# fill list of paths 
image_paths=[]
for i in range(n_images):
    image_paths.append('images\\panorama\\'+str(i+1)+'.jpg')
    
##### END STUDENT CODE
    
    
# fill list of images 
imgs = [] 
for i in range(n_images): 
    img = cv2.imread(image_paths[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # in case of memory error, potentially reduce image size
    width = int(img.shape[1]/2)
    height = int(img.shape[0]/2)
    img = cv2.resize(img, (width, height))
    
    imgs.append(img) 
     
# show the original pictures 
fig, axes = plt.subplots(1, n_images, figsize=(10, 30))
for i in range(n_images): 
    axes[i].imshow(imgs[i])  

## Panorama stitching - the fast way

Let's start by following the fast method to stitch a panorama using OpenCV's `Stitcher` class. Note that depending on your choice of single images, the stitching is not always successful. Also note that due to the underlying random sampling in RANSAC, the result is not deterministic, you might get different results for multiple code runs. 

In [ ]:
# create stitcher object and stitch images
stitchy=cv2.Stitcher.create() 
(dummy,output)=stitchy.stitch(imgs) 

# check if the stitching procedure was successful   
if dummy != cv2.STITCHER_OK: 
  # .stitch() returns a true value if stitching is  
  # done successfully 
    print("Stitching was't successful!") 
else:  
    print('Your panorama is ready :-)') 
  
    # final output 
    fig, ax = plt.subplots(1,1, figsize=(10, 30))
    ax.imshow(output) 

## Panorama stitching - step by step

Now let's again stitch a panorama without using the `Stitcher` class in order to understand what is happening under the hood. For the sake of simplicity, we only use the first two images for stitching. First we double the size of the second image to make room for the first image to be warped into this new image. The result looks as follows: 

In [ ]:
# create a blank image the same size as the second image - note that an image is just a numpy array here
blank_image = np.zeros((imgs[1].shape[0], imgs[1].shape[1], 3), np.uint8)

# horizontally concatenates images of same height  
imgs[1] = cv2.hconcat([blank_image, imgs[1]]) 
plt.imshow(imgs[1])

Task 2: SIFT Descriptors
=

Now we use SIFT to detect feature points and descriptors in both images and we plot the descriptors.

**TODO**: 2a) Use OpenCV's SIFT method to detect keypoints and descriptors of the two images. Plot the resulting keypoints and descriptors in the two images. **(4 points)**

**Hint:** You can use the example code from the lecture slides as reference.

In [ ]:
##### TODO: Instanciate SIFT detector


##### TODO: find the keypoints and descriptors with SIFT
 

In [ ]:
##### TODO: plot result 


**TODO**: 2b) Why do we use SIFT feature points and not Harris Corners for matching? **(2 points)**

**YOUR ANSWER:** 

Task 3: Matching Feature Points
=

**TODO**: 3a) Find matches between the two images using FLANN (Fast Library for Approximate Nearest Neighbors).  **(2 points)**

**Hint:** You can find a documentation of FLANN here: https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html 

In [ ]:
##### TODO: find matches with FLANN


**TODO**: 3b) Store good matches following Lowe's ratio test (see lecture slides). Again, you can use the documentation linked above. **(2 points)**

In [ ]:
##### TODO: store all the good matches in a list as per Lowe's ratio test.


**TODO**: 3c) Why do we look at two different distances in Lowe's ratio test? Why not simply use a threshold? **(2 points)**

**YOUR ANSWER:** 

Task 4: Model Fitting (RANSAC)
=

**TODO**: 4a) Use the matches to calculate a homography between the two images. Print the homography matrix. **(3 points)**

**Hint**: You can find a tutorial on RANSAC here: https://docs.opencv.org/4.x/d1/de0/tutorial_py_feature_homography.html

In [ ]:
##### TODO: YOUR CODE GOES HERE

    

**TODO:** 4b) Which of the 9 values in the homography matrix was not computed by RANSAC? Where does this value come from, and could you theoretically use another number instead?  **(2 points)**

**YOUR ANSWER:** 

**TODO**: 4c) Plot the two images and draw matches between features in the two images in red. Apply the homography to the boundary of the first image and plot the new boundary (after applying the homography) in blue into the second image.  **(4 points)**

In [ ]:
##### TODO: YOUR CODE GOES HERE


**TODO**: 4d) Finally, warp the first image into the second using the homography. You can use OpenCV's `warpPerspective()` to do this - look up the interface in the online documentation. Afterwards, blend the warped first image and the second image and plot the final panorama. **(2 points)**

**Hint:** To blend the two images, you can simply use the `add()` function as learned in the lesson. However, this will cause the overlapping area to be very light, because both pixel values add up. Alternatively, you can use `addWeighted()` with a blending factor `alpha` of 0.5. See the documentation here: https://docs.opencv.org/3.4/d5/dc4/tutorial_adding_images.html.

In [ ]:

##### TODO: Warp the first image using the homography


##### TODO: Blend the warped image with the second image using alpha blending


##### TODO: Display the blended image


That's it! You have just stitched your first panorama :-)

Task 5: Theory Questions
=

**TODO**: 5a) You have bought a new camera lens with a focal length of 50mm for your SLR camera. You know that the distance between the lens and the camera sensor is 6cm. How far away from the lens should you place an object to be in focus in the image? Explain your result either by writing the calculations down or by text. **(2.5 points)**

**YOUR ANSWER**: 


**TODO**: 5b) You are using a camera with intrinsic camera matrix $K=\begin{pmatrix}
2642 & 0 & 1034\\
0 & 2642 & 764\\
0 & 0 & 1
\end{pmatrix}$ (values given in pixels). What are the values of your 5 intrinsic parameters? **(2.5 points)**

**YOUR ANSWER**: 

**TODO**: 5c) You are taking a picture of an object located 10 meters away at the point $P=\begin{pmatrix}0\\0\\10\end{pmatrix}$ (in camera coordinates) with the camera from 5b). Which are the pixel coordinates of the corresponding image point? What is special about this point? **(2 points)**

**YOUR ANSWER**: 

**TODO**: 5d) Consider the image point $\begin{pmatrix}1034\\ 764 \end{pmatrix}$ in pixel coordinates. What can you tell about the corresponding real object that was projected onto this image point using the camera from 5b)? What do you know about its 3D camera coordinates? **(2 points)**

**YOUR ANSWER**: 